<center> 
    <h1> 
        Data Camp Project

 <center>
    <h1> 
        Prediction of the engagement rate on Instagram

# Table of content
1. [Introduction](#Introduction)
    - [Growing influence of Instagram](#See_later)
    - [Definition of the problem : Quantify the reaction to a post](#See_later)
2. [Data exploration](#Data_exploration)
    - [Data Processing](#Data_Processing)
    - [See later](#See_later)
    - [See later](#See_later)
3. [Workflow](#Workflow)
4. [Submission](#Submission)

# Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

# Introduction

## Growing influence of Instagram

The leading treads towards social networking services had drawn massive
public attention from last one and half decade. The amount of data that is
uploaded to these social networking services is increasing day by day. So,
there is massive requirement to study the highly dynamic behavior of users
towards these services. Let's now focus of one the most used social networks : Instagram 

Instagram is an American photo and video sharing social networking service. In 2018, Instagram has reached 1 billion monthly active users. 

## Definition of the problem : Quantify the reaction to a post

Aim : Predict the engagment rate by instagramers

Scoring : "Root mean square error" which is defined by the following formula : 

    

# Data exploration

In [23]:
df = pd.read_csv('cr7.csv')
df = df = df.iloc[:1072,:]
df.shape

(1072, 17)

## Data Processing

In [24]:
df.head(2)

,Links to profile,Links to publication_gen_info,Name Account/Influencer,Professional Activity,Nationality,Brand,Account description,Number of posts,Number of followers,Number of followings,Verified Status,Links to publication_pub_info,Post description,Post number of comments,Post number of likes,Posting date,Media Type
0,https://www.instagram.com/cristiano/,https://www.instagram.com/p/-1JhCoBpan/,Cristiano Ronaldo,Footballeur,Portugal,Non,Cristiano Ronaldo,2 982 publications,252 764 771,459,Vérifié,h,Dedication ✌️👌👍💪🏽☀️,5493.0,887 059,2015-12-03T12:56:22.000Z,Photo
1,https://www.instagram.com/cristiano/,https://www.instagram.com/p/-1dEuSBpet/,Cristiano Ronaldo,Footballeur,Portugal,Non,Cristiano Ronaldo,2 982 publications,252 764 771,459,Vérifié,h,Job done!👌👔☀️ @sacoorbrothers,8546.0,1 064 362,2015-12-03T15:47:15.000Z,Photo


In [25]:
df['Brand'] = df['Brand'].replace({'Non':0})
df['Verified Status'] = df['Verified Status'].replace({'Vérifié':0})
df['Media Type'] = df['Media Type'].replace({'Photo':0, 'Video':19})

In [26]:
df['Posting date'] = df['Posting date'].apply(lambda x : x.replace(':','-'))
df['Posting date'] = df['Posting date'].apply(lambda x : x.replace('T','-'))
df['Posting date'] = df['Posting date'].apply(lambda x : x.split('-'))

df['year of posting'] = df['Posting date'].apply(lambda x : x[0])
df['month of posting'] = df['Posting date'].apply(lambda x : x[1])
df['day of posting'] = df['Posting date'].apply(lambda x : x[2])
df['hour of posting'] = df['Posting date'].apply(lambda x : x[3])

df = df.drop(columns = ['Posting date'])

In [27]:
df.head(2)

,Links to profile,Links to publication_gen_info,Name Account/Influencer,Professional Activity,Nationality,Brand,Account description,Number of posts,Number of followers,Number of followings,Verified Status,Links to publication_pub_info,Post description,Post number of comments,Post number of likes,Media Type,year of posting,month of posting,day of posting,hour of posting
0,https://www.instagram.com/cristiano/,https://www.instagram.com/p/-1JhCoBpan/,Cristiano Ronaldo,Footballeur,Portugal,0,Cristiano Ronaldo,2 982 publications,252 764 771,459,0,h,Dedication ✌️👌👍💪🏽☀️,5493.0,887 059,0,2015,12,03,12
1,https://www.instagram.com/cristiano/,https://www.instagram.com/p/-1dEuSBpet/,Cristiano Ronaldo,Footballeur,Portugal,0,Cristiano Ronaldo,2 982 publications,252 764 771,459,0,h,Job done!👌👔☀️ @sacoorbrothers,8546.0,1 064 362,0,2015,12,03,15


# Workflow

The work flow is composed of two essential elements that make up the submission: the feature extractor and the regressor. The first allows both the preparation of initial data and the creation of new variables. The second, on the other hand, allows a supervised learning model to be trained so that the engagement rate on Instagram's post can be correctly predicted. This model is trained on a part of the base obtained from the feature extractor output, then is evaluated on the remaining part.

# Submission